Funktioniert und nachvollziehbar (Quelle: https://www.kaggle.com/code/syedjaferk/imdb-sentiment-classification-using-naive-bayes/notebook und https://medium.com/hackerdawn/imdb-review-sentiment-analysis-using-logistic-regression-d7878ee01947)

To-Does:
 - Imports müssen nochmal gecheckt werden welche wirklich gebraucht werden
 - Code muss auskommentiert werden

In [8]:
import re
import string
import pandas as pd
import nltk
import tensorflow_datasets as tfds
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize
from string import punctuation
from spellchecker import SpellChecker
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

from bs4 import BeautifulSoup
from tqdm import tqdm

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [9]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer

In [10]:
# Load the IMDb reviews dataset
data = tfds.load('imdb_reviews', split={'train': 'train', 'test': 'test'})

In [11]:
# Convert the data into pandas DataFrame and decode bytes to string
train_df = tfds.as_dataframe(data['train'])
test_df = tfds.as_dataframe(data['test'])

train_df['text'] = train_df['text'].apply(lambda x: x.decode('utf-8'))
test_df['text'] = test_df['text'].apply(lambda x: x.decode('utf-8'))

train_df['label'] = train_df['label'].replace({0: 'negative', 1: 'positive'})
test_df['label'] = test_df['label'].replace({0: 'negative', 1: 'positive'})

# Split the data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(train_df, train_df['label'], test_size=0.2, random_state=42, stratify=train_df['label'])

In [12]:
# Label encoding for the sentiment labels
le = LabelEncoder()
y_train = le.fit_transform(y_train)
y_test = le.transform(y_test)

In [13]:
print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

(20000, 2) (20000,)
(5000, 2) (5000,)


# Data Cleaning

In [14]:
spell = SpellChecker()
lemmatizer = WordNetLemmatizer()

# Way better runtime!
cachedStopWords = stopwords.words("english")


def transformations(dataframe):
    # HTML Tags removal
    dataframe['text'] = dataframe['text'].apply(lambda words: re.sub('<[^<]+?>', '', words))

    # Lower case conversion
    dataframe['text'] = dataframe['text'].apply(lambda words: words.lower())

    # Word Tokenization
    dataframe['text'] = dataframe['text'].apply(word_tokenize)

    # Punctuation removal
    dataframe['text'] = dataframe['text'].apply(lambda words: [x for x in words if not x in punctuation])

    # Number removal
    dataframe['text'] = dataframe['text'].apply(lambda words: [x for x in words if not x.isdigit()])

    # # Frequent word removal
    # temp = dataframe['text'].apply(lambda words: " ".join(words))
    # freq = pd.Series(temp).value_counts()[:10]
    # dataframe['text'] = dataframe['text'].apply(lambda words: [x for x in words if x not in freq.keys()])

    # Stopword removal
    dataframe['text'] = dataframe['text'].apply(lambda words: [x for x in words if x not in cachedStopWords])

    # Lemmatization
    dataframe['text'] = dataframe['text'].apply(lambda words: [lemmatizer.lemmatize(x) for x in words])

    # Join again
    dataframe['text'] = dataframe['text'].apply(lambda words: " ".join(words))

    return dataframe

In [15]:
x_train = transformations(x_train)

In [16]:
x_test = transformations(x_test)

# Vectorizer

In [17]:
from sklearn.feature_extraction.text import CountVectorizer

vec = CountVectorizer()
vec = vec.fit(x_train.text)
train_x_bow = vec.transform(x_train.text)
test_x_bow = vec.transform(x_test.text)

In [18]:
print(x_train.shape)
print(y_train.shape)

(20000, 2)
(20000,)


# Naive Bayes with Hyperparameter Tuning

In [19]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

# Assuming you have already defined the param_grid 'alpha_ranges' for 'C' values

# Create a Logistic Regression classifier
classifier = LogisticRegression(max_iter=100)

# Define the parameter grid for GridSearchCV
param_grid = {
    'C': [0.001, 0.01, 0.1, 1, 10.0, 100]  # Replace this list with your 'alpha_ranges' values for 'C'
}

# Create GridSearchCV object
grid_search = GridSearchCV(classifier, param_grid=param_grid, scoring='accuracy', cv=2, return_train_score=True)

# Fit the model on the training data
grid_search.fit(train_x_bow, y_train)

# Get the best estimator from the grid search
best_classifier = grid_search.best_estimator_

# Make predictions on the test data using the best estimator
predict = best_classifier.predict(test_x_bow)


c:\Users\User\OneDrive - METRO\Desktop\Uni\Dis25 - Haak\Exam\dis25_nlp\.venv\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\Users\User\OneDrive - METRO\Desktop\Uni\Dis25 - Haak\Exam\dis25_nlp\.venv\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative sol

In [20]:
from sklearn.metrics import accuracy_score
print("Accuracy is ", accuracy_score(y_test, predict))

Accuracy is  0.8908


In [21]:
from sklearn.metrics import classification_report
print("Accuracy is ", classification_report(y_test, predict))

Accuracy is                precision    recall  f1-score   support

           0       0.90      0.88      0.89      2500
           1       0.88      0.91      0.89      2500

    accuracy                           0.89      5000
   macro avg       0.89      0.89      0.89      5000
weighted avg       0.89      0.89      0.89      5000



In [23]:
from joblib import dump
dump(vec, 'vectors.logistic_regression')
dump(best_classifier, 'classifier.logistic_regression')

['classifier.logistic_regression']